### Basic Setup
Here we check the sage version, configure some settings, define a 2-manifold $M$ with cylidrical coordinate chart $Y$, and a riemannian metric. The naming convention python variables (not variables used in the actual expressions) is `CamelCase` for objects defined on the manifold, and `snake_case` (with a trailing underscore for functions) for generic symbolic expressions. For rank $(0, 2)$ tensors, we default to the purely covariant form (as that is what we have regularization conditions for), denote the $(1, 1)$ version as `X_mat` and denote the rank $(2, 0)$ version as `X_con`.

In [1]:
# Check version
version()

'SageMath version 9.5, Release Date: 2022-01-30'

In [2]:
# Reset
reset()
# Setup pretty printing
%display latex

In [3]:
# Define a differentiable manifold of dimension 2 over real numbers
M = Manifold(2, 'M', latex_name=r'\mathcal{M}', start_index=1)
# Define a cylindrical chart on the manifold
Chart.<rho, z> = M.chart(r'rho:\rho z:z')
# Cache frame
Chartf = Chart.frame()

In [4]:
# Declare a metric on the manifold
G = M.riemannian_metric('g')

# Components of metric
s_ = function('seed', latex_name='S')(rho, z)
psi_ = function('psi', latex_name=r'\psi')(rho, z)

# Metric is symmetric
# G[1, 1], G[2, 2] = psi_^4 * exp(2 * rho * s_), psi_^4 * exp(2 * rho * s_)
# G[1, 2] = 0

conformal_ = psi_**4 * exp(2 * rho * s_)

G[1, 1], G[2, 2] = conformal_, conformal_
G[1, 2] = 0

# Invert metric
G_con = G.inverse()

# Print metric
G.display()

g = e^(2*rho*seed(rho, z))*psi(rho, z)^4 drho⊗drho + e^(2*rho*seed(rho, z))*psi(rho, z)^4 dz⊗dz

In [5]:
G_con.display()

inv_g = e^(-2*rho*seed(rho, z))/psi(rho, z)^4 ∂/∂rho⊗∂/∂rho + e^(-2*rho*seed(rho, z))/psi(rho, z)^4 ∂/∂z⊗∂/∂z

In [6]:
# Compute Ricci Tensor
R = G.ricci(Chartf)
# And Ricci Scalar
R_trace = G_con['^ij'] * R['_ij']
# And connection coefficients, used for covariant derivative
Nabla = G.connection(name='nabla', latex_name=r'\nabla')
# Make sure metric is compatible
assert(Nabla(G) == 0)

Nabla.display()

Gam^rho_rho,rho = (rho*psi(rho, z)*d(seed)/drho + psi(rho, z)*seed(rho, z) + 2*d(psi)/drho)/psi(rho, z) 
Gam^rho_rho,z = (rho*psi(rho, z)*d(seed)/dz + 2*d(psi)/dz)/psi(rho, z) 
Gam^rho_z,rho = (rho*psi(rho, z)*d(seed)/dz + 2*d(psi)/dz)/psi(rho, z) 
Gam^rho_z,z = -(rho*psi(rho, z)*d(seed)/drho + psi(rho, z)*seed(rho, z) + 2*d(psi)/drho)/psi(rho, z) 
Gam^z_rho,rho = -(rho*psi(rho, z)*d(seed)/dz + 2*d(psi)/dz)/psi(rho, z) 
Gam^z_rho,z = (rho*psi(rho, z)*d(seed)/drho + psi(rho, z)*seed(rho, z) + 2*d(psi)/drho)/psi(rho, z) 
Gam^z_z,rho = (rho*psi(rho, z)*d(seed)/drho + psi(rho, z)*seed(rho, z) + 2*d(psi)/drho)/psi(rho, z) 
Gam^z_z,z = (rho*psi(rho, z)*d(seed)/dz + 2*d(psi)/dz)/psi(rho, z)

### Variables
Next we define the relavent tensor fields, as well as their components in our default frame.

In [7]:
# Lambda ($\lambda^2 = g_{\phi \phi}$ I believe)

# Resulting Tensors
Lam = M.scalar_field(rho * psi_^2, chart=Chart, name='lam', latex_name=r'\lambda')

LamLog = Nabla(Lam) / Lam
LamLog2 = Nabla(Nabla(Lam)) / Lam
           
Lam.display()

lam: M → ℝ
   (rho, z) ↦ rho*psi(rho, z)^2

In [8]:
# Gauge Fields
lapse_ = function('lapse', latex_name=r'\alpha')(rho, z)

shiftr_ = function('shiftr', latex_name=r'\beta^r')(rho, z)
shiftz_ = function('shiftz', latex_name=r'\beta^z')(rho, z)

# Gauge Tensors
Lapse = M.scalar_field(lapse_, chart=Chart, name='Lapse', latex_name=r'\alpha')

LapseLog = Nabla(Lapse) / Lapse
LapseLog2 = Nabla(Nabla(Lapse)) / Lapse

Shift = M.tensor_field(1, 0, name='Shift', latex_name=r'\beta')
Shift.add_comp(Chartf)[1] = shiftr_
Shift.add_comp(Chartf)[2] = shiftz_

Lapse.display()

Lapse: M → ℝ
   (rho, z) ↦ lapse(rho, z)

In [9]:
Shift.display()

Shift = shiftr(rho, z) ∂/∂rho + shiftz(rho, z) ∂/∂z

In [10]:
# Extrinsic Curvature
x_ = function('X')(rho, z)
u_ = function('U')(rho, z)
w_ = function('W')(rho, z)

# Original components are for $K^i_j$ so we have to scale by conformal factor to get $K_{ij}$.
krr_ = 1/3 * (rho * w_ - u_) * conformal_
kzz_ = 2/3 * (u_ + rho * w_ / 2) * conformal_
krz_ = x_ * conformal_

K = M.tensor_field(0, 2, sym=(0, 1), name='K')
K.add_comp(Chartf)[1, 1] = krr_
K.add_comp(Chartf)[2, 2] = kzz_
K.add_comp(Chartf)[1, 2] = krz_

K_mat = G_con['^{ij}'] * K['_jk']
K_con = K_mat['^i_k'] * G_con['^{kj}']
K_trace = K_mat.trace()

L = M.scalar_field(-K_trace.expr(), name='L', latex_name=r'L')

K_mat.display()

(1/3*rho*W(rho, z) - 1/3*U(rho, z)) ∂/∂rho⊗drho + X(rho, z) ∂/∂rho⊗dz + X(rho, z) ∂/∂z⊗drho + (1/3*rho*W(rho, z) + 2/3*U(rho, z)) ∂/∂z⊗dz

In [11]:
L.display()

L: M → ℝ
   (rho, z) ↦ -2/3*rho*W(rho, z) - 1/3*U(rho, z)

### Constraint equations
Constraint equations that should be satisfied at all times.

In [14]:
term1 = (K_trace^2 - K['_ij'] * K_con['^ij'] + R_trace) / 2
term2 = - LamLog2['_ij'] * G_con['^{ij}'] + K_trace * L

# Hamiltonian
CH = term1 + term2

view(CH.expr().simplify_full())

In [16]:
term1 = Nabla(K_mat)['^i_ji'] - Nabla(K_trace + L)
term2 = LamLog['_j'] * K_mat['^j_i'] - LamLog * L

# Momentum Constraint
CM = term1 + term2

latex(CM[1].expr().simplify_full())

-\frac{3 \, {\rho} U\left({\rho}, {z}\right) \psi\left({\rho}, {z}\right) \frac{\partial}{\partial {\rho}}S\left({\rho}, {z}\right) - 6 \, {\rho} X\left({\rho}, {z}\right) \psi\left({\rho}, {z}\right) \frac{\partial}{\partial {z}}S\left({\rho}, {z}\right) + 3 \, U\left({\rho}, {z}\right) \psi\left({\rho}, {z}\right) S\left({\rho}, {z}\right) - {\rho} \psi\left({\rho}, {z}\right) \frac{\partial}{\partial {\rho}}W\left({\rho}, {z}\right) - 4 \, W\left({\rho}, {z}\right) \psi\left({\rho}, {z}\right) + \psi\left({\rho}, {z}\right) \frac{\partial}{\partial {\rho}}U\left({\rho}, {z}\right) - 3 \, \psi\left({\rho}, {z}\right) \frac{\partial}{\partial {z}}X\left({\rho}, {z}\right) - 6 \, {\left({\rho} W\left({\rho}, {z}\right) - U\left({\rho}, {z}\right)\right)} \frac{\partial}{\partial {\rho}}\psi\left({\rho}, {z}\right) - 18 \, X\left({\rho}, {z}\right) \frac{\partial}{\partial {z}}\psi\left({\rho}, {z}\right)}{3 \, \psi\left({\rho}, {z}\right)}

In [20]:
CM[2].expr().simplify_full()

1/3*(6*rho^2*X(rho, z)*psi(rho, z)*diff(seed(rho, z), rho) + 3*rho^2*U(rho, z)*psi(rho, z)*diff(seed(rho, z), z) + 6*rho*X(rho, z)*psi(rho, z)*seed(rho, z) + rho^2*psi(rho, z)*diff(W(rho, z), z) + 2*rho*psi(rho, z)*diff(U(rho, z), z) + 3*rho*psi(rho, z)*diff(X(rho, z), rho) + 18*rho*X(rho, z)*diff(psi(rho, z), rho) + 3*X(rho, z)*psi(rho, z) + 6*(rho^2*W(rho, z) + 2*rho*U(rho, z))*diff(psi(rho, z), z))/(rho*psi(rho, z))

### Evolution Equations
Calculations for $\mathcal{L}_n X$ for various dynamical quantities.

In [15]:
term1 = R - LamLog2 - LapseLog2
term2 = (K_trace + L) * K - 2 * K['_ij'] * K_mat['^j_k']

# Extrinsic Curvature
LieK = term1 + term2

In [16]:
term1 = - LamLog2['_ij'] * G_con['^{ij}']
term2 = - (LamLog * LapseLog)['_{ij}'] * G_con['^{ij}']
term3 = L * (K_trace + L)

# Angular Extrinsic Curvature
LieL = term1 + term2 + term3

In [17]:
# Metric and Lambda
LieG = -2*K
LieLam = -Lam * L